# 🎥AI-Powered Videos/ lecture Summarization
> **Transform hours of video into minutes of reading.**

### 📖 Overview
**How it works:**
1. installing transformer
2. extract text from video
3. summarization pipeline
4. summrized text generation


# Installing Transformer 

In [1]:
!pip install transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 103.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.


# Extract text from video

In [2]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 9.6 MB/s eta 0:00:00


In [3]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')

    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

if __name__ == "__main__":
    url = "https://www.youtube.com/watch?v=rVfZHWTwXSA"

    video_id = extract_video_id(url)

    api = YouTubeTranscriptApi()
    fetched = api.fetch(video_id, languages=['en'])

    text = "\n".join(snippet.text for snippet in fetched)

In [4]:
text

'All right. [NOISE] Um, let\'s get started.\nSo, um, let\'s see, logistical reminder,\nuh the class midterm, um,\nis this Wednesday and it\'s 48-hour take-home midterm.\nUm, and the logistical details you can find,\nuh, at this Piazza post, okay?\nSo the midterm will start Wednesday evening.\nYou have 48 hours to do it and then submit it online through Gradescope, uh,\nand because of the midterm,\nthere won\'t be a section,\nuh, this Friday, okay?\nOh and the midterm will cover everything up to and including EM,\nuh, which we\'ll spend most of today talking about, okay?\nCertainly don\'t look so stressed. It\'ll be fun.\n[LAUGHTER].\nMaybe. All right.\nUm, so what I\'d like to do today is start our foray into, uh, unsupervised learning.\nUh, so far I\'ve spent a lot of time on\nsupervised learning algorithms including advice on\nhow to apply supervised learning algorithms.\nThese pens are great.\nIn which you\'d have, you know,\npositive examples and negative examples and you run\nlogi

# load summrizing pipeline
* the used model [facebook/bart-large-cnn] limit is *1024 tokens*.
* so we will be using *chunking*
* By spliting the long lecture/video into small pieces
* summarizes each piece, and gives you a full summary.

In [5]:
# Run this cell FIRST every time you start a new session!
!pip install -U transformers accelerate

# Only uncomment the line below if you are starting a fresh session
# import os
# os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0
    Uninstalling transformers-4.38.0:
      Successfully uninstalled transformers-4.38.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [6]:
from transformers import pipeline
import math

# Load the model on GPU
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

#  function that split text into chunks
def chunk_text(text, max_chunk_size=3000):
    
    # 3000 characters is roughly 700-800 tokens, which is safe for BART (limit 1024)
    chunks = []
    for i in range(0, len(text), max_chunk_size):
        chunks.append(text[i:i + max_chunk_size])
    return chunks

# 3. Process the long video transcript
print("Summarizing video...")
chunks = chunk_text(text)
full_summary = []

for i, chunk in enumerate(chunks):
    # Skip very short chunks (like the very end)
    if len(chunk) < 100:
        continue
        
    print(f"Processing chunk {i+1}/{len(chunks)}...")
    
    # Summarize this specific chunk
    # We use truncation=True just in case, but our manual splitting prevents the crash
    summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False, truncation=True)
    full_summary.append(summary[0]['summary_text'])

# 4. Combine and print the final result
print("\n--- FINAL VIDEO SUMMARY ---\n")
final_text = " ".join(full_summary)
print(final_text)

2026-01-01 15:48:08.703134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767282488.906305      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767282488.961165      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767282489.441083      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767282489.441137      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767282489.441140      24 computation_placer.cc:177] computation placer alr

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summarizing video...
Processing chunk 1/19...
Processing chunk 2/19...
Processing chunk 3/19...
Processing chunk 4/19...
Processing chunk 5/19...
Processing chunk 6/19...
Processing chunk 7/19...
Processing chunk 8/19...
Processing chunk 9/19...
Processing chunk 10/19...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing chunk 11/19...
Processing chunk 12/19...
Processing chunk 13/19...
Processing chunk 14/19...
Processing chunk 15/19...
Processing chunk 16/19...
Processing chunk 17/19...
Processing chunk 18/19...
Processing chunk 19/19...

--- FINAL VIDEO SUMMARY ---

The class midterm is this Wednesday and it's 48-hour take-home midterm. The first unsupervised learning algorithm we'll talk about is clustering. The most common uses of clustering is, uh, market segmentation. A clustering algorithm can be used to find clusters of dots near each other. The color of the dots depends on which cluster centroid is closer. The algorithm converges even if you keep running the algorithm. The algorithm is the same either way. It turns out that the algorithm can be proven to converge. The algorithm is written out in the lecture notes, but it turns out if you write it out as a function, it will cost less. The most frequently asked question I get for k-means is how do you choose k? It turns out that in u